# RT2 assignment II 
create a jupyter notebook to replace the add user interface for the assignment 2 of rt1. This interface will provide the user the capability of:
- assign (or cancel) goals to the robot
- plot with the robot’s position and targets’ positions in the environment
- plot for the number of reached/not-reached targets

## Inizialization
Import all the necessary modules

In [1]:
%matplotlib widget
# Import necessary modules
import ipywidgets as widgets # Library for the use of widget
import numpy as np 
import matplotlib.pyplot as plt # Library for use the script of matplotlib
from matplotlib.animation import FuncAnimation # Library for using matplotlib with python
from IPython.display import display

import rospy
import tf
from tf.transformations import quaternion_matrix

import actionlib
import assignment_2_2023
import assignment_2_2023.msg
from nav_msgs.msg import Odometry
from assignment_2_2023.msg import PlanningAction, PlanningActionGoal, RobotInfo, PlanningGoal, PlanningActionResult

# defining global variables
global last_x, last_y
global goal_x, goal_y
goal_x = None
goal_y = None
last_x = None
last_y = None

global reached_goal, not_reached_goal
reached_goal = []
not_reached_goal = []


## User interface
In this part is possible to send a new target at the robot or delete the actual target. Is possible to write the new target inside the box and use the bottom below for send or delete the message.

In the first two cells there all all teh code referred to the class and the function that I will use inide the node. In the  third script there is all the implementation of the node 

In [2]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.goal_ln, = plt.plot([], [], 'go')  # object for goals
        self.x_data, self.y_data = [] , []
        self.goal_x_data, self.goal_y_data = [] , []  # Data for goals

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.goal_ln,

    def odom_callback(self, msg):
        global goal_x, goal_y
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        
        # Get the position from the message
        position = msg.pose.pose.position
        # Update the text widget values
        x_position_display.value = position.x
        y_position_display.value = position.y
        
        if goal_x is not None and goal_y is not None:
            #print('ciao')
            if (abs(position.x - goal_x) <= 0.5) and (abs(position.y - goal_y) <= 0.5):
                reached_goal.append([goal_x, goal_y])
                #print('reached goal odom_callback: ')
                #print (reached_goal)

            
    

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.goal_ln.set_data(self.goal_x_data, self.goal_y_data)  # Update goal data
        return self.ln, self.goal_ln,

    def add_goal(self, x, y):
        self.goal_x_data.append(x)
        self.goal_y_data.append(y)

    def remove_goal(self, x, y):
        if x in self.goal_x_data and y in self.goal_y_data:
            index = self.goal_x_data.index(x)
            self.goal_x_data.pop(index)
            self.goal_y_data.pop(index)


In [3]:
# section with inside all the function thaht will be utilized in the code

# Define a function to handle button click (Submit Coordinates)
def on_submit_button_clicked(b):
    x = x_input.value
    y = y_input.value
    
    global last_x
    global last_y
    global reached_goal
    global not_reached_goal
    
    # Remopve the previon goal printed isnoide the graph
    # Check if there are last submitted coordinates
    if last_x is not None and last_y is not None:
        vis.remove_goal(last_x, last_y)
        print(f'Removed goal at: X={last_x}, Y={last_y}')
        
    # chek if the goal is reached or not, if it is reached append to the reached gloal list, if not append in the nor_reached list
    if interface.get_state() == actionlib.GoalStatus.SUCCEEDED:
        reached_goal.append([last_x, last_y])
    else:
        not_reached_goal.append([last_x, last_y])
    
    print('reached goal: ')
    print (reached_goal)
    print(f'not reached goal: ')
    print (not_reached_goal)
                
    # Here you can add the code to send the coordinates to the robot
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    # update the last coordinate of the goal
    last_x = x
    last_y = y
    # Send goal to the action server
    interface.send_goal(goal)
    vis.add_goal(x, y)
    print(f'Coordinates submitted: X={x}, Y={y}')

    
# Define a function to handle button click (Delete Coordinates)
def on_delete_button_clicked(b):
    
    global last_x
    global last_y
    global reached_goal
    global not_reached_goal
    
    if interface.get_state() == actionlib.GoalStatus.ACTIVE:
        interface.cancel_goal()
        print(f'Goal cancelled')
    else:
        print(f'There is no active goal to cancel')
     
    # Check if there are last submitted coordinates
    if last_x is not None and last_y is not None:
        vis.remove_goal(last_x, last_y)
        print(f'Removed goal at: X={last_x}, Y={last_y}') 
        
    # chek if the goal is reached or not, if it is reached append to the reached gloal list, if not append in the nor_reached list
    if interface.get_state() == actionlib.GoalStatus.SUCCEEDED:
        reached_goal.append([last_x, last_y])
    else:
        not_reached_goal.append([last_x, last_y])


# Function for update the bargraph
def update(num):
    ax.clear()
    ax.bar(['Reached Goal', 'Not Reached Goal'], [len(reached_goal), len(not_reached_goal)])
    ax.set_xlabel('Goal Status')
    ax.set_ylabel('Count')
    ax.set_title('Barplot of Goal Status')


## Jupyter node

In the cell below there is all the implementation of the node. 

In this section will be printed in a box text the position of the robot. For doing that I decide to subscribe to a topic /odom and print the value of the position of the robot. Those values are inside the mesage of type: msg.pose.pose.position and respectively on position x, y cordinates. 


In [4]:
%matplotlib widget

rospy.init_node("jupyter_node")

# Inizialize the client service
interface = actionlib.SimpleActionClient("/reaching_goal", assignment_2_2023.msg.PlanningAction)
interface.wait_for_server()
goal = assignment_2_2023.msg.PlanningGoal()

# Create text input fields for coordinates
x_input = widgets.FloatText(value=0.0, description='X Coordinate:')
y_input = widgets.FloatText(value=0.0, description='Y Coordinate:')

# Create a button to submit coordinates and delete coordinates
submit_button = widgets.Button(description='Submit Coordinates')
delete_button = widgets.Button(description='Delete Coordinates')

# Attach the functions to the buttons create before
submit_button.on_click(on_submit_button_clicked)
delete_button.on_click(on_delete_button_clicked)

# Create FloatText widgets for displaying robot position
x_position_display = widgets.FloatText(description='X:')
y_position_display = widgets.FloatText(description='Y:')

# Organizza i widget in un layout
top_box = widgets.HBox([x_input, y_input])
left_box = widgets.VBox([submit_button, delete_button])
bottom_box = widgets.HBox([x_position_display, y_position_display])
main_box = widgets.VBox([top_box, left_box, bottom_box])

# Mostra i widget
display(main_box)


# Display the widgets and use the FuncAnimation
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)


# Crea una figura per il grafico
fig, ax = plt.subplots()
# Func animation for the bargraph
ani1 = FuncAnimation(fig, update)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

reached goal: 
[]
not reached goal: 
[[None, None]]
Coordinates submitted: X=2.0, Y=1.0
